In [1]:
import pandas as pd
import seaborn as sns
from urllib.request import urlretrieve
import re

table_url = f'https://huggingface.co/datasets/poloclub/diffusiondb/resolve/main/metadata.parquet'
urlretrieve(table_url, 'metadata.parquet')

# Read the table using Pandas
metadata_df = pd.read_parquet('metadata.parquet')

In [2]:
metadata_df

,image_name,prompt,part_id,seed,step,cfg,sampler,width,height,user_name,timestamp,image_nsfw,prompt_nsfw
0,2217ccbd-a1c6-47ac-9a2d-79649727c834.png,"a portrait of a female robot made from code, v...",1,2615452544,50,7.0,8,512,512,0167cc490c3dc6047be855b096a9b9d8f20e2b034532bd...,2022-08-20 11:56:00+00:00,0.554853,0.001621
1,c78807b7-d55a-4a2d-a6b6-9192b18941ad.png,a portrait of a female robot made from a cloud...,1,4231506957,50,7.0,8,512,512,0167cc490c3dc6047be855b096a9b9d8f20e2b034532bd...,2022-08-20 11:59:00+00:00,0.153645,0.000707
2,dc71658a-5e4b-4dca-861a-e1535510348b.png,"only memories remain, trending on artstation",1,544607824,50,7.0,8,512,512,0167cc490c3dc6047be855b096a9b9d8f20e2b034532bd...,2022-08-20 12:02:00+00:00,0.062496,0.000425
3,48eb7e17-a3cf-4eb8-96a9-d8e3e23fa1af.png,dream swimming pool with nobody,1,3500509234,50,7.0,8,512,512,019084defdd44d2dfcd200bf7aaae53034263168e10d6b...,2022-08-20 07:22:00+00:00,0.030799,0.003586
4,601d9792-eccd-4850-97a7-edbe91d3464c.png,a dog doing weights. epic oil painting.,1,1312381211,50,12.0,8,512,768,03cb377ebf364ea2761b5710625974c0d2d25285fec982...,2022-08-20 12:28:00+00:00,0.181035,0.030822
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1999995,4a34854e-e117-4b69-b1da-cc74534524c0.png,david bowie giving a piggy back ride to ziggy ...,2000,3003198679,50,7.0,8,512,512,fff4138cde262b5632a036752b40ddec8cb0a53c549123...,2022-08-16 04:51:00+00:00,0.152736,0.022676
1999996,d4ae6d4f-762c-42f0-9481-a1c233e6a541.png,david bowie giving a piggy back ride to ziggy ...,2000,3003198679,50,7.0,8,512,512,fff4138cde262b5632a036752b40ddec8cb0a53c549123...,2022-08-16 04:53:00+00:00,0.127884,0.014115
1999997,67dac228-3631-4036-8498-a40a67a7d354.png,funny computer,2000,3602197422,50,7.0,8,512,512,fff4138cde262b5632a036752b40ddec8cb0a53c549123...,2022-08-16 05:41:00+00:00,0.093476,0.000463
1999998,9d57179b-de12-460b-a699-a66dfca528be.png,hilarious witty computing machine,2000,1604720580,50,7.0,8,512,512,fff4138cde262b5632a036752b40ddec8cb0a53c549123...,2022-08-16 06:47:00+00:00,0.124379,0.002547


In [3]:

# Our harm dictionary
# Organized as so: [HarmScore, Violent (0-1), Explicit (0-1)] 
harm_dict = {
    "porn": [10, 0, 1],
    "nude": [8, 0, 1],
    "erotic": [6, 0, 1],
    "bondage": [8, 0, 1],
    "boobs": [5, 0, 1],
    "creampie": [6, 0, 1],
    "fisting": [8, 1, 1],
    "gangbang": [7, 1, 1],
    "gore": [8, 1, 0],
    "hentai": [10, 0, 1],
    "lolicon": [10, 0, 1],
    "moaning": [5, 0, 1],
    "nsfw": [8, 0, 1],
    "sex": [10, 0, 1],
    "sexual": [8, 0, 1],
    "shota": [10, 0, 1],
    "tits": [6, 0, 1],
    "milf": [8, 0, 1],
    "anal": [7, 0, 1],
    "asshole": [6, 0, 1],
    "bukkake": [7, 1, 1],
    "cock": [6, 0, 1],
    "cum": [6, 0, 1],
    "cumming": [6, 0, 1],
    "cunnilingus": [7, 0, 1],
    "dildo": [6, 0, 1],
    "ejaculation": [6, 0, 1],
    "fellatio": [7, 0, 1],
    "footjob": [6, 0, 1],
    "incest": [10, 1, 1],
    "necrophilia": [10, 1, 1],
    "paizuri": [7, 0, 1],
    "pedophilia": [10, 1, 1],
    "rape": [10, 1, 1],
    "rape fantasy": [10, 1, 1],
    "sadomasochism": [8, 1, 1],
    "scat": [9, 1, 1],
    "snuff": [10, 1, 1],
    "tentacle": [7, 0, 1],
    "zoophilia": [10, 1, 1],
    "fursona": [10, 0, 1],
    "attractive": [3, 0, 0],
    "emma watson": [7, 0, 0],
    "full shot": [2, 0, 0],
    "beautiful": [1, 0, 0],
    "pretty": [1, 0, 0],
    "sexy": [2, 0, 0],
    "hot": [2, 0, 0],
    "sexy": [2, 0, 0],
    "submissive": [2, 0, 0],
    "sensual": [2, 0, 0],

}

innoTerms = {
    "woman": [2, 0, 0],
    "girl": [4, 0, 0],
    "teen": [4, 0, 0],
    "child": [4, 0, 0],
    "kid": [4, 0, 0],
    "teenager": [4, 0, 0],
    "boy": [4, 0, 0],
    "lolita": [10, 0, 0],
    "minor": [5, 0, 0],
    "preteen": [5, 0, 0],
    "underage": [6, 0, 0],
    "adolescent": [4, 0, 0],
    "young": [3, 0, 0],
    "juvenile": [4, 0, 0],
    "infant": [5, 0, 0],
    "toddler": [5, 0, 0],
    "schoolgirl": [5, 0, 0],
    "schoolboy": [5, 0, 0],
    "baby": [4, 0, 0],
    "hot": [2, 0, 0],
    "beautiful": [1, 0, 0],
    "swimsuit": [2, 0, 0],
    "swimsuits": [2, 0, 0],
    "swimwear": [2, 0, 0],
    "swimsuit": [2, 0, 0],
    "dress": [2, 0, 0],
}



def get_prompt_harm_score(prompt, harm_map, inno_map):
    tokens = prompt.lower().split()
    
    # 1) Check if there's at least one harm_dict token in the prompt
    has_harm = any(token in harm_map for token in tokens)
    
    # 2) If none found, ignore innoTerms and return 0
    if not has_harm:
        return 0
    
    # 3) Otherwise, sum up scores from BOTH dictionaries
    total_score = 0
    for token in tokens:
        # Harm dict score
        harm_score = harm_map.get(token, [0, 0, 0])[0]
        # Inno dict score
        inno_score = inno_map.get(token, [0, 0, 0])[0]
        
        total_score += harm_score
        total_score += inno_score
    
    return total_score



# Calculate harm scores for each prompt
metadata_df['harm_score'] = metadata_df['prompt'].apply(
    lambda x: get_prompt_harm_score(x, harm_dict, innoTerms)
)

print("Finished")


Finished


In [4]:
metadata_df['harm_score'].max()

np.int64(46)

In [5]:
print(metadata_df)

                                       image_name  \
0        2217ccbd-a1c6-47ac-9a2d-79649727c834.png   
1        c78807b7-d55a-4a2d-a6b6-9192b18941ad.png   
2        dc71658a-5e4b-4dca-861a-e1535510348b.png   
3        48eb7e17-a3cf-4eb8-96a9-d8e3e23fa1af.png   
4        601d9792-eccd-4850-97a7-edbe91d3464c.png   
...                                           ...   
1999995  4a34854e-e117-4b69-b1da-cc74534524c0.png   
1999996  d4ae6d4f-762c-42f0-9481-a1c233e6a541.png   
1999997  67dac228-3631-4036-8498-a40a67a7d354.png   
1999998  9d57179b-de12-460b-a699-a66dfca528be.png   
1999999  0ae508b1-f140-48c9-9da4-a124967215cf.png   

                                                    prompt  part_id  \
0        a portrait of a female robot made from code, v...        1   
1        a portrait of a female robot made from a cloud...        1   
2            only memories remain, trending on artstation         1   
3                         dream swimming pool with nobody         1   
4       

In [6]:
filteredData = metadata_df.sort_values(by="harm_score", ascending=False)

In [7]:
filteredData

,image_name,prompt,part_id,seed,step,cfg,sampler,width,height,user_name,timestamp,image_nsfw,prompt_nsfw,harm_score
172230,d82b8a74-370e-416d-b192-0aa8463d2c56.png,A mechanical-steampunk themed lolita outfit ; ...,173,3491905876,50,7.0,8,512,512,373c26cf546e435d5025be53574419c4322c509555e750...,2022-08-09 05:39:00+00:00,0.148281,0.000387,46
659394,63aef8d4-9eaf-4e2f-8e4c-376f5d9916f3.png,gorgeous werefox fursona fursona fursona furso...,660,4207515083,50,7.0,8,512,512,57efd5d70bac55912a133b7349b6ca9b27f60c34c2cba3...,2022-08-11 03:46:00+00:00,0.162564,0.019261,40
707893,5223fe0a-3e83-4bcc-92e2-24cbb1580bbe.png,psychedelic organic pregnant cyborg made of cl...,708,1940607077,120,10.0,8,512,640,eadd7a1448d7a8c2b44db43c2124d6ebb46530dc6752c9...,2022-08-10 03:59:00+00:00,0.853388,0.008826,36
1404789,ca58035b-d8c1-460f-8003-46151117e628.png,1 6 k neon - monochrome 3 d of cgsociety : - n...,1405,3324874849,50,11.0,8,512,768,deleted_acount,2022-08-13 11:57:00+00:00,0.793580,0.276294,32
1404788,f5dbea23-0133-4861-95ea-43144bdce0ce.png,1 6 k neon - monochrome 3 d of cgsociety : - n...,1405,2265294034,50,11.0,8,512,768,deleted_acount,2022-08-13 11:39:00+00:00,0.801473,0.085923,32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11,a02dba37-321a-48f8-989e-915e79dd66f1.png,concept art of a silent hill monster. painted ...,1,2055775158,50,6.0,8,640,512,08ac4153ab7e13baa55ee74213eb117a29a0024267b29e...,2022-08-20 05:22:00+00:00,0.075712,0.083516,0
12,01419f71-766d-4d50-9e63-e880d0d1961e.png,concept art of a silent hill monster. painted ...,1,2291766792,50,5.0,8,640,512,08ac4153ab7e13baa55ee74213eb117a29a0024267b29e...,2022-08-20 05:24:00+00:00,0.076030,0.083516,0
13,e912ddad-85a4-44df-9b16-c888b02bd126.png,concept art of a silent hill monster. painted ...,1,4233354762,50,5.0,8,640,512,08ac4153ab7e13baa55ee74213eb117a29a0024267b29e...,2022-08-20 05:26:00+00:00,0.067348,0.083516,0
14,6722011e-92e6-4523-b05e-52c5785c364a.png,an emo portrait painting. short dark brown mes...,1,423272642,50,6.0,8,512,512,08ac4153ab7e13baa55ee74213eb117a29a0024267b29e...,2022-08-20 06:25:00+00:00,0.117506,0.043428,0


In [ ]:
def detect_terms_with_synergy(prompt, harm_dict, inno_dict):
    """
    Returns a tuple (harm_terms_found, inno_terms_found) for a single prompt.
    'inno_terms_found' is only populated if there's at least one harm term.
    """
    prompt_lower = prompt.lower()
    
    # 1. Find all harm terms in this prompt
    harm_terms_found = []
    for term in harm_dict.keys():
        pattern = rf"\b{re.escape(term)}\b"
        if re.search(pattern, prompt_lower):
            harm_terms_found.append(term)
    
    # 2. If no harm term is present, skip inno detection
    if not harm_terms_found:
        return harm_terms_found, []
    
    # 3. Otherwise, find inno terms too
    inno_terms_found = []
    for term in inno_dict.keys():
        pattern = rf"\b{re.escape(term)}\b"
        if re.search(pattern, prompt_lower):
            inno_terms_found.append(term)
    
    return harm_terms_found, inno_terms_found

# Now apply row by row
metadata_df["harm_terms_found"], metadata_df["inno_terms_found"] = zip(
    *metadata_df["prompt"].apply(lambda x: detect_terms_with_synergy(x, harm_dict, innoTerms))
)

In [ ]:
metadata_df

,image_name,prompt,part_id,seed,step,cfg,sampler,width,height,user_name,timestamp,image_nsfw,prompt_nsfw,harm_score,prompt_lower,harm_terms_found,inno_terms_found
0,2217ccbd-a1c6-47ac-9a2d-79649727c834.png,"a portrait of a female robot made from code, v...",1,2615452544,50,7.0,8,512,512,0167cc490c3dc6047be855b096a9b9d8f20e2b034532bd...,2022-08-20 11:56:00+00:00,0.554853,0.001621,0,"a portrait of a female robot made from code, v...",[],[]
1,c78807b7-d55a-4a2d-a6b6-9192b18941ad.png,a portrait of a female robot made from a cloud...,1,4231506957,50,7.0,8,512,512,0167cc490c3dc6047be855b096a9b9d8f20e2b034532bd...,2022-08-20 11:59:00+00:00,0.153645,0.000707,0,a portrait of a female robot made from a cloud...,[],[]
2,dc71658a-5e4b-4dca-861a-e1535510348b.png,"only memories remain, trending on artstation",1,544607824,50,7.0,8,512,512,0167cc490c3dc6047be855b096a9b9d8f20e2b034532bd...,2022-08-20 12:02:00+00:00,0.062496,0.000425,0,"only memories remain, trending on artstation",[],[]
3,48eb7e17-a3cf-4eb8-96a9-d8e3e23fa1af.png,dream swimming pool with nobody,1,3500509234,50,7.0,8,512,512,019084defdd44d2dfcd200bf7aaae53034263168e10d6b...,2022-08-20 07:22:00+00:00,0.030799,0.003586,0,dream swimming pool with nobody,[],[]
4,601d9792-eccd-4850-97a7-edbe91d3464c.png,a dog doing weights. epic oil painting.,1,1312381211,50,12.0,8,512,768,03cb377ebf364ea2761b5710625974c0d2d25285fec982...,2022-08-20 12:28:00+00:00,0.181035,0.030822,0,a dog doing weights. epic oil painting.,[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1999995,4a34854e-e117-4b69-b1da-cc74534524c0.png,david bowie giving a piggy back ride to ziggy ...,2000,3003198679,50,7.0,8,512,512,fff4138cde262b5632a036752b40ddec8cb0a53c549123...,2022-08-16 04:51:00+00:00,0.152736,0.022676,0,david bowie giving a piggy back ride to ziggy ...,[],[]
1999996,d4ae6d4f-762c-42f0-9481-a1c233e6a541.png,david bowie giving a piggy back ride to ziggy ...,2000,3003198679,50,7.0,8,512,512,fff4138cde262b5632a036752b40ddec8cb0a53c549123...,2022-08-16 04:53:00+00:00,0.127884,0.014115,0,david bowie giving a piggy back ride to ziggy ...,[],[]
1999997,67dac228-3631-4036-8498-a40a67a7d354.png,funny computer,2000,3602197422,50,7.0,8,512,512,fff4138cde262b5632a036752b40ddec8cb0a53c549123...,2022-08-16 05:41:00+00:00,0.093476,0.000463,0,funny computer,[],[]
1999998,9d57179b-de12-460b-a699-a66dfca528be.png,hilarious witty computing machine,2000,1604720580,50,7.0,8,512,512,fff4138cde262b5632a036752b40ddec8cb0a53c549123...,2022-08-16 06:47:00+00:00,0.124379,0.002547,0,hilarious witty computing machine,[],[]


In [ ]:
filteredData2 = metadata_df.sort_values(by="harm_score", ascending=False)
filteredData2

: 

## Muitiplication Factor(Term) / Total (Term[s]) 

### Scalability depending on the term T1, T2, T3, T4 Terms. If T1 term is combined with T2 Term = T4. Ex. "Lolita" & "T2" = Likely CP -> T4